# Codebase RAG Project

![Screenshot 2024-11-25 at 7 12 58 PM](https://github.com/user-attachments/assets/0bd67cf0-43d5-46d2-879c-a752cae4c8e3)

# Install Necessary Libraries

In [1]:
! pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers

  Using cached langchain_core-0.3.79-py3-none-any.whl.metadata (3.2 kB)
Using cached langchain_core-0.3.79-py3-none-any.whl (449 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.1
    Uninstalling langchain-core-1.0.1:
      Successfully uninstalled langchain-core-1.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 3.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from langchain_core.pydantic_v1 import BaseModel
from pinecone import Pinecone

/usr/local/lib/python3.12/dist-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


# Clone a GitHub Repo locally

In [4]:
def clone_repository(repo_url):
    """Clones a GitHub repository to a temporary directory.

    Args:
        repo_url: The URL of the GitHub repository.

    Returns:
        The path to the cloned repository.
    """
    repo_name = repo_url.split("/")[-1]  # Extract repository name from URL
    repo_path = f"/content/{repo_name}"
    if not os.path.exists(repo_path):
        Repo.clone_from(repo_url, str(repo_path))
    return str(repo_path)

In [5]:
path = clone_repository("https://github.com/Yejin-Hwang/Stock-Prediction-using-Transformer")

In [6]:
print(path)

/content/Stock-Prediction-using-Transformer


In [7]:
SUPPORTED_EXTENSIONS = {'.py', '.js', '.tsx', '.jsx', '.ipynb', '.java',
                         '.cpp', '.ts', '.go', '.rs', '.vue', '.swift', '.c', '.h'}

IGNORED_DIRS = {'node_modules', 'venv', 'env', 'dist', 'build', '.git',
                '__pycache__', '.next', '.vscode', 'vendor'}

In [8]:
def get_file_content(file_path, repo_path):
    """
    Get content of a single file.

    Args:
        file_path (str): Path to the file

    Returns:
        Optional[Dict[str, str]]: Dictionary with file name and content
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        # Get relative path from repo root
        rel_path = os.path.relpath(file_path, repo_path)

        return {
            "name": rel_path,
            "content": content
        }
    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        return None


def get_main_files_content(repo_path: str):
    """
    Get content of supported code files from the local repository.

    Args:
        repo_path: Path to the local repository

    Returns:
        List of dictionaries containing file names and contents
    """
    files_content = []

    try:
        for root, _, files in os.walk(repo_path):
            # Skip if current directory is in ignored directories
            if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
                continue

            # Process each file in current directory
            for file in files:
                file_path = os.path.join(root, file)
                if os.path.splitext(file)[1] in SUPPORTED_EXTENSIONS:
                    file_content = get_file_content(file_path, repo_path)
                    if file_content:
                        files_content.append(file_content)

    except Exception as e:
        print(f"Error reading repository: {str(e)}")

    return files_content

In [9]:
file_content = get_main_files_content(path)

In [10]:
file_content

[{'name': 'Amazon-chronos model_TSLA.ipynb',
  'content': '{\n "cells": [\n  {\n   "cell_type": "markdown",\n   "metadata": {},\n   "source": [\n    "## Amazon chronos-t5-small models for predicting stock\'s price\\n",\n    "\\n",\n    "https://huggingface.co/amazon/chronos-t5-small"\n   ]\n  },\n  {\n   "cell_type": "code",\n   "execution_count": 1,\n   "metadata": {},\n   "outputs": [],\n   "source": [\n    "# %pip install git+https://github.com/amazon-science/chronos-forecasting.git\\n",\n    "# !pip install transformers"\n   ]\n  },\n  {\n   "cell_type": "code",\n   "execution_count": 2,\n   "metadata": {},\n   "outputs": [],\n   "source": [\n    "import yfinance as yf\\n",\n    "import torch\\n",\n    "import numpy as np\\n",\n    "import pandas as pd\\n",\n    "import matplotlib.pyplot as plt\\n",\n    "from chronos import ChronosPipeline"\n   ]\n  },\n  {\n   "cell_type": "markdown",\n   "metadata": {},\n   "source": [\n    "# Dataset "\n   ]\n  },\n  {\n   "cell_type": "code",\

# Embeddings

In [11]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [12]:
text = "I am a programmer"

embeddings = get_huggingface_embeddings(text)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
embeddings

array([ 1.81737654e-02, -3.02657508e-03, -4.77465875e-02,  1.86379403e-02,
        3.14537995e-02,  1.87255293e-02, -1.52534274e-02, -6.77293688e-02,
       -1.26903653e-02,  1.28427576e-02,  5.80701306e-02,  4.00234833e-02,
        3.27073298e-02,  7.12998435e-02,  5.56373373e-02,  1.68628506e-02,
        6.97603747e-02, -5.02619930e-02,  6.13140827e-03, -1.46559235e-02,
       -4.51957993e-03,  4.82934639e-02, -2.53051296e-02, -1.97862904e-03,
       -4.36902530e-02, -2.41507161e-02,  1.29505759e-02, -3.78611824e-03,
       -2.05718316e-02,  1.09819308e-01,  3.07672890e-03, -2.80443169e-02,
       -1.55807249e-02, -1.24789868e-02,  1.75239131e-06, -2.93756695e-03,
       -1.43048428e-02,  4.88386713e-02, -6.21114224e-02,  2.95061413e-02,
       -1.40470508e-02,  2.20708270e-02,  1.13067888e-02,  4.70893271e-02,
        7.58305984e-03, -8.30314530e-05,  6.67821169e-02, -1.21320095e-02,
        4.39386303e-03,  2.47453637e-02,  1.02529004e-02, -6.54432410e-03,
       -5.53147821e-03, -

# Setting up Pinecone
**1. Create an account on [Pinecone.io](https://app.pinecone.io/)**

**2. Create a new index called "codebase-rag" and set the dimensions to 768. Leave the rest of the settings as they are.**

![Screenshot 2024-11-24 at 10 58 50 PM](https://github.com/user-attachments/assets/f5fda046-4087-432a-a8c2-86e061005238)



**3. Create an API Key for Pinecone**

![Screenshot 2024-11-24 at 10 44 37 PM](https://github.com/user-attachments/assets/e7feacc6-2bd1-472a-82e5-659f65624a88)


**4. Store your Pinecone API Key within Google Colab's secrets section, and then enable access to it (see the blue checkmark)**

![Screenshot 2024-11-24 at 10 45 25 PM](https://github.com/user-attachments/assets/eaf73083-0b5f-4d17-9e0c-eab84f91b0bc)



In [14]:
# Set the PINECONE_API_KEY as an environment variable
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index("codebase-rag")

In [15]:
vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())

/tmp/ipython-input-2416920381.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())
/tmp/ipython-input-2416920381.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())


In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

documents = []

for file in file_content:
    doc = Document(
        page_content=f"{file['name']}\n{file['content']}",
        metadata={"source": file['name']}
    )

    documents.append(doc)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)

# chunked_documents = text_splitter.split_documents(documents)

# print(f"Number of original documents: {len(documents)}")
# print(f"Number of chunked documents: {len(chunked_documents)}")

Number of original documents: 3
Number of chunked documents: 321


In [17]:
# Explanation of the error:
# The previous attempt to upsert documents to Pinecone failed with a PineconeApiException and a "message length too large" error.
# This is because the size of the combined data being sent in a single upsert request (the list of original documents) exceeded Pinecone's maximum message size limit (4194304 bytes).
# To fix this, we need to split the large documents into smaller chunks before upserting them. This has already been done in the previous step, resulting in the 'chunked_documents' list.

# Remove the failed upsert attempt code (already commented out in the previous turn, but explicitly noting its removal here)
# The code cell 'tDAB_siIb93B' contained the failed upsert attempt and will not be re-executed.


# Embed the generated text chunks and upsert them into the Pinecone index
vectorstore = PineconeVectorStore.from_documents(
    documents=chunked_documents,
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/msmarco-distilbert-base-v4"),
    index_name="codebase-rag",
    namespace="https://github.com/Yejin-Hwang/Stock-Prediction-using-Transformer",
    batch_size=100 # Use the same batch_size as in the previous attempt
)

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)

In [19]:
for i, doc in enumerate(documents):
    print(f"Document {i}: {len(doc.page_content)} bytes")

Document 0: 95348 bytes
Document 1: 64560 bytes
Document 2: 93506 bytes


In [20]:
stats = pinecone_index.describe_index_stats()
print(stats)

{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'https://github.com/CoderAgent/SecureAgent': {'vector_count': 52},
                'https://github.com/Yejin-Hwang/Financial-Time-Series-Forecasting-with-Deep-Learning-Models-and-Social-Media-Sentiment': {'vector_count': 13902},
                'https://github.com/Yejin-Hwang/Stock-Prediction-using-Transformer': {'vector_count': 642}},
 'total_vector_count': 14596,
 'vector_type': 'dense'}


In [21]:
from pinecone import Pinecone
import os
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index = pc.Index("codebase-rag")

stats = index.describe_index_stats()
print("namespaces:", list(stats.get("namespaces", {}).keys()))
print(stats)


namespaces: ['https://github.com/CoderAgent/SecureAgent', 'https://github.com/Yejin-Hwang/Stock-Prediction-using-Transformer', 'https://github.com/Yejin-Hwang/Financial-Time-Series-Forecasting-with-Deep-Learning-Models-and-Social-Media-Sentiment']
{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'https://github.com/CoderAgent/SecureAgent': {'vector_count': 52},
                'https://github.com/Yejin-Hwang/Financial-Time-Series-Forecasting-with-Deep-Learning-Models-and-Social-Media-Sentiment': {'vector_count': 13902},
                'https://github.com/Yejin-Hwang/Stock-Prediction-using-Transformer': {'vector_count': 642}},
 'total_vector_count': 14596,
 'vector_type': 'dense'}


# Perform RAG

1. Get your Groq API Key [here](https://console.groq.com/keys)

2. Paste your Groq API Key into your Google Colab secrets, and make sure to enable permissions for it

![Screenshot 2024-11-25 at 12 00 16 AM](https://github.com/user-attachments/assets/e5525d29-bca6-4dbd-892b-cc770a6b281d)


In [22]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=userdata.get("GROQ_API_KEY")
)

In [84]:
query = "which evaluation matrix was used in chronos model? "

In [85]:
raw_query_embedding = get_huggingface_embeddings(query)

raw_query_embedding

array([-1.90068930e-02, -5.31988218e-02, -8.42371769e-03,  5.31592080e-03,
       -2.03919690e-02,  2.15852968e-02,  7.81693235e-02,  2.53566690e-02,
       -1.97134372e-02,  6.97354088e-03,  1.61342956e-02, -1.05195853e-03,
        4.91970219e-03,  4.00003046e-02, -2.20774254e-03,  2.71048546e-02,
        4.68622409e-02,  2.38504037e-02, -8.41938928e-02,  1.50250588e-02,
       -2.63910722e-02, -4.70786765e-02, -1.65990542e-03,  1.11562219e-02,
        3.06401066e-02, -2.02244353e-02, -1.47456732e-02,  2.49067671e-03,
        1.36114499e-02, -3.99534442e-02,  1.39965571e-03,  6.50830194e-02,
        4.70613316e-02, -9.55795404e-03,  1.14063789e-06, -2.33283360e-02,
       -4.15858114e-03,  3.51987220e-02,  7.89368618e-03,  8.00440758e-02,
        9.35358927e-02,  5.44631779e-02,  7.81537127e-03, -5.28816059e-02,
       -2.70063747e-02, -1.19183501e-02, -1.34123452e-02,  2.69551342e-03,
       -3.19972187e-02, -1.17676202e-02,  1.98781230e-02,  5.77646755e-02,
        9.28618712e-04,  

In [86]:
# Feel free to change the "top_k" parameter to be a higher or lower number
top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=5, include_metadata=True, namespace="https://github.com/Yejin-Hwang/Stock-Prediction-using-Transformer")

In [87]:
top_matches

{'matches': [{'id': '6ad65a40-5b45-43cb-9019-ab0357fc8070',
              'metadata': {'source': 'ARIMA_TSLA.ipynb',
                           'text': '" ARIMA(0,1,0)(2,0,0)[12]             : '
                                   'AIC=3575.868, Time=0.12 sec\\n",\n'
                                   '      " '
                                   'ARIMA(0,1,0)(2,0,2)[12]             : '
                                   'AIC=3574.615, Time=0.65 sec\\n",\n'
                                   '      " '
                                   'ARIMA(1,1,0)(1,0,1)[12]             : '
                                   'AIC=3574.182, Time=0.21 sec\\n",\n'
                                   '      " '
                                   'ARIMA(0,1,1)(1,0,1)[12]             : '
                                   'AIC=3574.192, Time=0.21 sec\\n",\n'
                                   '      " '
                                   'ARIMA(1,1,1)(1,0,1)[12]             : '
                             

In [88]:
contexts = [item['metadata']['text'] for item in top_matches['matches']]

In [89]:
contexts

['" ARIMA(0,1,0)(2,0,0)[12]             : AIC=3575.868, Time=0.12 sec\\n",\n      " ARIMA(0,1,0)(2,0,2)[12]             : AIC=3574.615, Time=0.65 sec\\n",\n      " ARIMA(1,1,0)(1,0,1)[12]             : AIC=3574.182, Time=0.21 sec\\n",\n      " ARIMA(0,1,1)(1,0,1)[12]             : AIC=3574.192, Time=0.21 sec\\n",\n      " ARIMA(1,1,1)(1,0,1)[12]             : AIC=3576.173, Time=0.27 sec\\n",\n      "\\n",\n      "Best model:  ARIMA(0,1,0)(1,0,1)[12]          \\n",\n      "Total fit time: 8.367 seconds\\n"\n     ]\n    }\n   ],\n   "source": [\n    "from pmdarima import auto_arima\\n",\n    "\\n",\n    "stepwise_fit = auto_arima(train[\'Close\'],\\n",\n    "                          start_p=1, start_q=1,\\n",\n    "                          max_p=5, max_q=5,\\n",\n    "                          d=1,\\n",\n    "                          seasonal=True,\\n",\n    "                          m=12,  # 월별 주기면 12\\n",\n    "                          trace=True,\\n",\n    "                      

In [90]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [91]:
print(augmented_query)

<CONTEXT>
" ARIMA(0,1,0)(2,0,0)[12]             : AIC=3575.868, Time=0.12 sec\n",
      " ARIMA(0,1,0)(2,0,2)[12]             : AIC=3574.615, Time=0.65 sec\n",
      " ARIMA(1,1,0)(1,0,1)[12]             : AIC=3574.182, Time=0.21 sec\n",
      " ARIMA(0,1,1)(1,0,1)[12]             : AIC=3574.192, Time=0.21 sec\n",
      " ARIMA(1,1,1)(1,0,1)[12]             : AIC=3576.173, Time=0.27 sec\n",
      "\n",
      "Best model:  ARIMA(0,1,0)(1,0,1)[12]          \n",
      "Total fit time: 8.367 seconds\n"
     ]
    }
   ],
   "source": [
    "from pmdarima import auto_arima\n",
    "\n",
    "stepwise_fit = auto_arima(train['Close'],\n",
    "                          start_p=1, start_q=1,\n",
    "                          max_p=5, max_q=5,\n",
    "                          d=1,\n",
    "                          seasonal=True,\n",
    "                          m=12,  # 월별 주기면 12\n",
    "                          trace=True,\n",
    "                          stepwise=True)"
   ]
  },
  

In [92]:
system_prompt = f"""You are a Senior Data Scientist, specializing in Python.

Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
"""

llm_response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]
)

response = llm_response.choices[0].message.content

In [93]:
response

'The provided code snippet does not directly mention how the best model was evaluated, but it does hint at using some form of model comparison. Looking through the code, it can be inferred that the model comparison or evaluation is likely based on the Akaike information criterion (AIC). The AIC is a metric used in model selection that balances the model\'s ability to fit the data with the number of parameters required.\n\nHere\'s a brief snippet that implies the usage of AIC within the Chronos Model:\n\n`" ARIMA(0,1,0)(2,0,0)[12]             : AIC=3575.868, Time=0.12 sec\\n"`\n\nHowever, the actual Chronos model is a more extensive library compared to what is provided in the code snippet, and as such, the actual evaluation metric might be beyond simple AIC. \n\nThe answer to this, therefore, is not clear from the information provided within the snippet.'

# Putting it all together

In [94]:
def perform_rag(query, image_input=None):
    # TODO: Handle image input and potentially combine with text query for multimodal embedding
    raw_query_embedding = get_huggingface_embeddings(query)

    # Reduced top_k to decrease the amount of context sent to the LLM
    top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=1, include_metadata=True, namespace="https://github.com/Yejin-Hwang/Stock-Prediction-using-Transformer")

    # Get the list of retrieved texts
    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

    # Modify the prompt below as need to improve the response quality
    system_prompt = f"""You are a Senior Software Engineer, specializing in TypeScript.

    Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
    """

    llm_response = client.chat.completions.create(
        model="qwen/qwen3-32b", # NOTE: This model was suggested in the previous turn.
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )

    return llm_response.choices[0].message.content

In [95]:
response = perform_rag("which evaluation matrix was used in chronos model")

print(response)

<think>
Okay, the user is asking which evaluation matrix was used in the Chronos model. Let me start by looking at the provided context.

First, the context shows the output from an ARIMA model trained using pmdarima's auto_arima function. The lines list different ARIMA models with their AIC values and fitting times. The best model selected is ARIMA(0,1,0)(1,0,1)[12]. The source code uses the 'auto_arima' function with parameters like start_p, start_q, max_p, max_q, etc., which are typical in autoregressive models. 

The user mentioned "Chronos model," but the context doesn't explicitly mention Chronos. However, the code and output are related to ARIMA. Since ARIMA is a traditional time series model, I need to clarify if Chronos refers to a specific framework or another model. But given the context, it's likely the user is referring to the ARIMA model selection process here. 

In the context provided, the evaluation metric used is AIC (Akaike Information Criterion). The models are comp

**Reasoning**:
The task is to modify the `perform_rag` function to accept an optional image input. I will modify the function signature to accept an optional `image_input` parameter and add a placeholder comment inside the function to indicate where the image processing logic will be added in the next steps.

